### 환경 install

In [1]:
#pip install langchain-google-genai


In [2]:
#pip install langchain langchain-community langchain-core langchain-text-splitters langchain-google-genai faiss-cpu pymupdf

In [3]:
#pip install langchain-postgres psycopg2-binary

In [4]:
#pip install google-generativeai

In [5]:
# pip install langchain langchain-community langchain-openai

In [6]:
# pip install langchain-openai

In [7]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_community.vectorstores import FAISS
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import PromptTemplate
# from langchain_community.document_loaders import TextLoader
# from langchain_community.vectorstores import PGVector
# # from langchain_community.retrievers import MergerRetriever  

# # 변경 부분 - Google genai, and PGVector
# from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# from langchain_postgres.vectorstores import PGVector
# from langchain_community.document_loaders import DirectoryLoader, TextLoader
# import time

import os
import pandas as pd
import numpy as np
import openai
from datetime import datetime
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import PGVector
from dotenv import load_dotenv
# from langchain_community.retrievers import MergerRetriever  
from langchain_openai import ChatOpenAI

# 변경 부분 - Google genai, and PGVector
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_postgres.vectorstores import PGVector
from langchain_community.document_loaders import DirectoryLoader, TextLoader
import time
from dotenv import load_dotenv


#Openai 추가
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector

In [9]:
from langchain_community.docstore import InMemoryDocstore
from langchain_classic.retrievers import TimeWeightedVectorStoreRetriever


#셀프쿼리시 추가 library
from langchain_core.structured_query import Comparator, Operator
from langchain_classic.retrievers.self_query.pgvector import PGVectorTranslator
from langchain_classic.retrievers import SelfQueryRetriever
from langchain_classic.chains.query_constructor.schema import AttributeInfo
from langchain_classic.retrievers import EnsembleRetriever
from kiwipiepy import Kiwi
from langchain_core.retrievers import BaseRetriever
from typing import List
from langchain_core.documents import Document

#hybrid 추가 library
from typing import List, Any, Optional
from pydantic import ConfigDict

In [10]:
# API KEY 체크
load_dotenv(override=True)

# API 키 확인
api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키: {api_key[:8]}")
CONNECTION_STRING = os.getenv("DATABASE_URL")
print(f"DB connection string : {CONNECTION_STRING[:3]}")

# 만약 None이면 문제!
if api_key is None:
    print("❌ .env 파일에 OPENAI_API_KEY가 없습니다!")

if CONNECTION_STRING is None:
    print("❌ .env 파일에 CONNECTION_STRING가 없습니다!")

API 키: sk-proj-


## 임베딩 Ai 연결, DB 연결

In [11]:
# 단계 3: 임베딩(Embedding) 생성
#openai 사용 
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.


# 임베딩 추가 없이 DB만 호출하고 싶을때는 아래 코드 활성화  
vectorstore = PGVector(
    embeddings=embeddings,
    collection_name="SI_try",
    connection=CONNECTION_STRING,  # connection_string이 아니라 connection
)


In [12]:
# # langchain 소스코드 확인하기
# # 흥미로운 내용 많네요 
# ??PGVector


# 일반 유사도 검색(기존)

In [13]:
# 코사인 거리(cosine distance) = 1 - cos(θ)
# - 범위: 0 ~ 2 (보통 0 ~ 1)
# - 0에 가까울수록 유사

#k는 문서 개수, 정의 안하면 기본 4개 
results = vectorstore.similarity_search_with_score("투썸플레이스 영수증프린터", k=10)
for i, (doc, score) in enumerate(results, 1):
    print(f"{i}위 - 유사도: {score:.4f}")
    print(f"내용: {doc.page_content[:50]}...")
    print("---")

1위 - 유사도: 0.4128
내용: [접수번호 26440]
접수일시: 2025-05-23 20:21
가맹점: 투썸플레이스 - ...
---
2위 - 유사도: 0.4323
내용: [접수번호 15386]
접수일시: 2025-03-24 10:03
가맹점: 투썸플레이스 - ...
---
3위 - 유사도: 0.4364
내용: [접수번호 7572]
접수일시: 2025-02-11 19:58
가맹점: 투썸플레이스 - 투...
---
4위 - 유사도: 0.4400
내용: [접수번호 14579]
접수일시: 2025-03-19 11:44
가맹점: 투썸플레이스 - ...
---
5위 - 유사도: 0.4406
내용: [접수번호 25969]
접수일시: 2025-05-21 12:18
가맹점: 투썸플레이스 - ...
---
6위 - 유사도: 0.4430
내용: [접수번호 39310]
접수일시: 2025-08-03 11:44
가맹점: 투썸플레이스 - ...
---
7위 - 유사도: 0.4459
내용: [접수번호 67808]
접수일시: 2025-12-22 16:08
가맹점: 투썸플레이스 - ...
---
8위 - 유사도: 0.4481
내용: [접수번호 42562]
접수일시: 2025-08-22 09:45
가맹점: 투썸플레이스 - ...
---
9위 - 유사도: 0.4536
내용: [접수번호 40300]
접수일시: 2025-08-08 10:40
가맹점: 투썸플레이스 - ...
---
10위 - 유사도: 0.4541
내용: [접수번호 17333]
접수일시: 2025-04-02 13:15
가맹점: 투썸플레이스 - ...
---


In [14]:
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.99, k=1
)



# retriever.invoke("투썸플레이스 영수증프린터")

# 방법 1: similarity_search로 확인
# all_docs = vectorstore.similarity_search("", k=100)
# print(f"저장된 문서 수 (최대 100개): {len(all_docs)}")

# for i, doc in enumerate(all_docs[:5]):
#     print(f"\n문서 {i+1}:")
#     print(f"내용: {doc.page_content[:100]}...")
#     print(f"메타데이터: {doc.metadata}")

# 기본 similarity_search로 테스트
basic_results = vectorstore.similarity_search("투썸플레이스 영수증프린터", k=5)
print(f"기본 검색 결과: {len(basic_results)}개\n")

for i, doc in enumerate(basic_results):
    print(f"결과 {i+1}:")
    print(f"내용: {doc.page_content[:150]}")
    print(f"메타데이터: {doc.metadata}\n")

기본 검색 결과: 5개

결과 1:
내용: [접수번호 26440]
접수일시: 2025-05-23 20:21
가맹점: 투썸플레이스 - 투썸플레이스(당진시청점)

문제 요약: 영수증프린터 출력 안됨

상세 내용:
영수증 출력 안됨

처리 결과:
용지교체, 셀프테스트 후 정상.

분류: 장비 > 프린터(영수증,주방)
메타데이터: {'no': 26440, 'date': '2025-05-23', 'type': '원격', 'year': 2025, 'month': 5, 'summary': '영수증프린터 출력 안됨', 'category': '투썸플레이스', 'store_name': '투썸플레이스(당진시청점)', 'sub_category': '프린터(영수증,주방)', 'main_category': '장비'}

결과 2:
내용: [접수번호 15386]
접수일시: 2025-03-24 10:03
가맹점: 투썸플레이스 - 투썸(인천검단금호점)

문제 요약: 영수증 출력 관련

상세 내용:
영수증 프린터 출력안됨

처리 결과:
용지교체, 내부청소, 방전했으나 동일 커팅이 잘 안되어 칼날조절 후 정상

메타데이터: {'no': 15386, 'date': '2025-03-24', 'type': '원격', 'year': 2025, 'month': 3, 'summary': '영수증 출력 관련', 'category': '투썸플레이스', 'store_name': '투썸(인천검단금호점)', 'sub_category': '프린터(영수증,주방)', 'main_category': '장비'}

결과 3:
내용: [접수번호 7572]
접수일시: 2025-02-11 19:58
가맹점: 투썸플레이스 - 투썸플레이스(청주비하점)

문제 요약: 영수증 출력 안 됨

상세 내용:
영수증 출력 안 됨

처리 결과:
프린터 전원 켠 후 정상

분류: 장비 > 프린터(영수증,주방)
처리 방식
메타데이터: {'no': 7572, 'date': '2025-02-11', 'type': '원격', 'year': 2025, 'm

In [15]:
# def get_recent_relevant_docs(query, k=5):
#     """유사도 검색 후 최신순 정렬"""
#     # 충분한 후보 가져오기
#     results = vectorstore.similarity_search(query, k=k*2)
    
#     # 날짜순 정렬
#     results_with_date = [doc for doc in results if doc.metadata.get('date')]
#     results_with_date.sort(
#         key=lambda x: x.metadata['date'], 
#         reverse=True  # 최신순
#     )
    
#     return results_with_date[:k]

# # 사용
# results = get_recent_relevant_docs("투썸플레이스 영수증프린터", k=5)

# for doc in results:
#     print(f"📅 {doc.metadata['date']} | {doc.metadata['store_name']}")
#     print(f"   {doc.page_content[:100]}...\n")

In [16]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List, Any
from datetime import datetime
from pydantic import Field, ConfigDict

class DateWeightedRetriever(BaseRetriever):
    """날짜 가중치를 적용한 커스텀 Retriever"""
    
    vectorstore: Any = Field(exclude=True)
    decay_days: int = 30
    k: int = 5
    sort_by: str = 'score'
    
    # ✅ Pydantic V2 방식
    model_config = ConfigDict(arbitrary_types_allowed=True)
    
    def _get_relevant_documents(
        self, 
        query: str, 
        *, 
        run_manager: CallbackManagerForRetrieverRun | None = None
    ) -> List[Document]:
        """관련 문서 검색"""
        candidates = self.vectorstore.similarity_search_with_score(query, k=self.k*3)
        scored_docs = []
        now = datetime.now()
        
        for doc, similarity_score in candidates:
            if doc.metadata.get('date'):
                doc_date = datetime.strptime(doc.metadata['date'], '%Y-%m-%d')
                days_old = (now - doc_date).days
                time_weight = max(0, 1 - (days_old / self.decay_days))
                final_score = similarity_score * 0.7 + time_weight * 0.3
                scored_docs.append((doc, final_score, doc_date))
        
        if self.sort_by == 'date':
            scored_docs.sort(key=lambda x: x[2], reverse=True)
        else:
            scored_docs.sort(key=lambda x: x[1], reverse=True)
        
        return [doc for doc, score, date in scored_docs[:self.k]]

# Retriever 생성
retriever = DateWeightedRetriever(
    vectorstore=vectorstore,
    decay_days=90,
    k=5,
    sort_by='date'
)

# 테스트
results = retriever.invoke("투썸플레이스 영수증프린터")
for doc in results:
    print(f"📅 {doc.metadata['date']} | {doc.metadata['store_name']}")
    print(f"   {doc.page_content[:100]}...\n")

📅 2025-12-22 | 투썸플레이스(서동탄역점)
   [접수번호 67808]
접수일시: 2025-12-22 16:08
가맹점: 투썸플레이스 - 투썸플레이스(서동탄역점)

문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)

상세...

📅 2025-12-21 | 투썸플레이스(동서울터미널점)
   [접수번호 67490]
접수일시: 2025-12-21 09:50
가맹점: 투썸플레이스 - 투썸플레이스(동서울터미널점)

문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)

...

📅 2025-10-13 | 투썸(동탄역앨리스빌점)
   [접수번호 53284]
접수일시: 2025-10-13 09:35
가맹점: 투썸플레이스 - 투썸(동탄역앨리스빌점)

문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)

상세 ...

📅 2025-08-22 | 투썸플레이스(구리역점)
   [접수번호 42562]
접수일시: 2025-08-22 09:45
가맹점: 투썸플레이스 - 투썸플레이스(구리역점)

문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)

상세 ...

📅 2025-08-08 | 투썸플레이스(대구연경점)
   [접수번호 40300]
접수일시: 2025-08-08 10:40
가맹점: 투썸플레이스 - 투썸플레이스(대구연경점)

문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)

상세...



In [17]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
# retriever = vectorstore.as_retriever()

# Retriever 생성
retriever = DateWeightedRetriever(
    vectorstore=vectorstore,
    decay_days=90,
    k=10,
    sort_by='date'  # 또는 'score'
)


# # mmr로 바꾸고 싶을 때
# retriever = vectorstore.as_retriever(search_type="mmr")
# print(retriever.search_type)  


In [18]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("투썸플레이스 영수증프린터")

[Document(id='c8c4256c-cca1-4431-9e6e-f488c1be7a42', metadata={'no': 67931, 'date': '2025-12-23', 'type': '원격', 'year': 2025, 'month': 12, 'summary': 'POS프린터 출력이상(흐림, 잘림, 안나옴)', 'category': '투썸플레이스', 'store_name': '투썸(인천영종하늘중앙로점)', 'sub_category': '프린터(영수증,주방)', 'main_category': '장비'}, page_content='[접수번호 67931]\n접수일시: 2025-12-23 09:11\n가맹점: 투썸플레이스 - 투썸(인천영종하늘중앙로점)\n\n문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)\n\n상세 내용:\n1번포스 영수증 출력 안됨.\n\n처리 결과:\n셀프테스트 프로그램 재실행 후 정상.\n\n분류: 장비 > 프린터(영수증,주방)\n처리 방식: 원격\n담당자: 우민정'),
 Document(id='78d4143d-2c13-4494-a829-74ae1b8ed00c', metadata={'no': 67808, 'date': '2025-12-22', 'type': '원격', 'year': 2025, 'month': 12, 'summary': 'POS프린터 출력이상(흐림, 잘림, 안나옴)', 'category': '투썸플레이스', 'store_name': '투썸플레이스(서동탄역점)', 'sub_category': '프린터(영수증,주방)', 'main_category': '장비'}, page_content='[접수번호 67808]\n접수일시: 2025-12-22 16:08\n가맹점: 투썸플레이스 - 투썸플레이스(서동탄역점)\n\n문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)\n\n상세 내용:\n2번 포스 영수증 출력 안됨\n\n처리 결과:\n셀프테스트 후 정상\n\n분류: 장비 > 프린터(영수증,주방)\n처리 방식: 원격\n담당자:

In [19]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for AS call center worker who needs information for error case - answer tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Provide several cases including details and then one summary sentence. 
Answer in Korean.

IMPORTANT: Ignore documents without exact match, Priority is user's query. Strictly follow the query.


#Context: 
{context}

#Question:
{question}

#Answer:"""
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 기존 체인 그대로 사용
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [20]:
from langchain_core.prompts import ChatPromptTemplate

In [21]:
# answer_prompt = ChatPromptTemplate.from_messages([
#     ("system", """You are a customer service case analysis expert. 
# Based on the retrieved consultation cases, provide your answer in the following format:

# 1. Present **5 cases** that match the query in the following format:
#    - Store Name: 
#    - Franchise Category:
#    - Receipt Date:
#    - Issue Summary:
#    - Detailed Content:
#    - Resolution Result:

# 2. At the end, provide a brief **overall analysis** of all cases

# **IMPORTANT: Respond in Korean, even though these instructions are in English.**
# """),
#     ("user", """Question: {question}

# Search Results:
# {context}

# Please answer according to the format above, in Korean.""")
# ])

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | answer_prompt
#     | llm
#     | StrOutputParser()
# )

In [22]:
# 단계 8: 체인(Chain) 생성
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

In [23]:
%%time
question = "투썸 매장에서 POS 프린터 오류 어떻게 처리했어? 접수일자랑 같이 알려줘"
response = chain.invoke(question)
print(f"💬 답변:\n{response}")

💬 답변:
투썸 매장에서 POS 프린터 오류 처리 사례는 다음과 같습니다:

1. **투썸 중앙대점**
   - **접수일자**: 2025-12-28
   - **문제 요약**: POS프린터 출력이상(흐림, 잘림, 안나옴)
   - **상세 내용**: 배달전용 포스 연결된 프린터기 출력 안됨. 피드 버튼 누를 시 아무 반응 없음.
   - **처리 결과**: 처리중

2. **투썸(금천롯데캐슬골드파크점)**
   - **접수일자**: 2025-12-01
   - **문제 요약**: POS프린터 출력이상(흐림, 잘림, 안나옴)
   - **상세 내용**: 주문서 출력 안 됨.
   - **처리 결과**: 랜선 재연결 후 정상, PC 이용 안내.

3. **투썸 광주상무메이루즈(2)**
   - **접수일자**: 2025-11-09
   - **문제 요약**: POS프린터 출력이상(흐림, 잘림, 안나옴)
   - **상세 내용**: 주문서 출력 안 됨.
   - **처리 결과**: 매장 아님. 증상 전달받아 매장에서 접수 안내.

4. **투썸 강남도산(2)**
   - **접수일자**: 2025-10-26
   - **문제 요약**: POS프린터 출력이상(흐림, 잘림, 안나옴)
   - **상세 내용**: 픽업프린터 에러 불 들어오며 출력 안 됨.
   - **처리 결과**: 처리중.

5. **투썸플레이스(통영죽림점)**
   - **접수일자**: 2025-10-22
   - **문제 요약**: POS프린터 출력이상(흐림, 잘림, 안나옴)
   - **상세 내용**: 영수증 프린터 글자 흐리게 출력 됨.
   - **처리 결과**: 처리중.

요약하자면, 투썸 매장에서 POS 프린터 오류는 주로 출력이 안 되거나 흐리게 나오는 문제로 접수되었으며, 일부는 원격으로 해결되었고, 다른 경우는 출장 서비스가 필요했습니다.
CPU times: total: 62.5 ms
Wall time: 16.7 s


In [24]:
%%time
question = "1월에서 2월 사이 투썸플레이스 키오스크 문의사항을 보여줘. 접수일자랑 같이"
response = chain.invoke(question)
print(f"💬 답변:\n{response}")

💬 답변:
죄송하지만, 1월에서 2월 사이의 투썸플레이스 키오스크 문의사항에 대한 정보는 없습니다.
CPU times: total: 46.9 ms
Wall time: 1.44 s


In [25]:
import tiktoken




# 1. 인코더 초기화 (GPT-4o-mini는 cl100k_base 사용)
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# 2. 각 부분의 토큰 수 계산
def count_tokens(text):
    return len(encoding.encode(text))

# 3. 실제 측정
question = "투썸 매장에서 POS 프린터 오류 어떻게 처리했어?"

# Context 가져오기
retrieved_docs = retriever.invoke(question)
context = "\n".join([doc.page_content for doc in retrieved_docs])

In [26]:
# 토큰 수, 가격 계산
# 모델 마다 토큰 당 가격이 다름 : gpt-4o-mini의 경우 1M token 당 input $0.15	캐시 input $0.075	output$0.60
# gpt-4o-mini price
input_price = 0.15/1000000
output_price = 0.6/1000000

# 환율
d2won = 1458

prompt_tokens = count_tokens(prompt.template)  # 템플릿 고정 부분
context_tokens = count_tokens(context)
question_tokens = count_tokens(question)
output_tokens = count_tokens(response)

total_input_tokens = prompt_tokens + context_tokens + question_tokens
total_output_tokens = output_tokens

total_token_price = total_input_tokens * input_price + output_tokens * output_price
Kwon_total_token_price = d2won * total_token_price

print(f"📊 토큰 사용량:")
print(f"  입력 토큰: {total_input_tokens:,}")
print(f"    - Prompt: {prompt_tokens}")
print(f"    - Context: {context_tokens}")
print(f"    - Question: {question_tokens}")
print(f"  출력 토큰: {total_output_tokens:,}")
print(f"  총 토큰: {total_input_tokens + total_output_tokens:,}")
# 유효숫자 2개
print(f"총 가격 ($) : {total_token_price:.2g}")
print(f"총 가격 (원) : {Kwon_total_token_price:.2g}")


📊 토큰 사용량:
  입력 토큰: 2,872
    - Prompt: 99
    - Context: 2758
    - Question: 15
  출력 토큰: 30
  총 토큰: 2,902
총 가격 ($) : 0.00045
총 가격 (원) : 0.65


In [27]:
#한달에 드는 총 비용 
# 주어진 데이터
monthly_mean = 5840.5  # 한 달 평균 건수
monthly_std = 651.9    # 표준편차
questions_per_case = 1  # 1건당 질문 수
cost_per_question = Kwon_total_token_price  # 1 질문당 비용 (원)

# 계산
total_questions_mean = monthly_mean * questions_per_case
total_questions_std = monthly_std * questions_per_case

total_cost_mean = total_questions_mean * cost_per_question
total_cost_std = total_questions_std * cost_per_question

print("=" * 60)
print("💰 한 달 LLM 비용 계산")
print("=" * 60)
print(f"월 평균 케이스:        {monthly_mean:,.1f}건")
print(f"1건당 질문 수:         {questions_per_case}번")
print(f"1질문당 비용:          {cost_per_question}원")
print()
print(f"월 총 질문 수:         {total_questions_mean:,.1f}회")
print(f"                       (±{total_questions_std:,.1f}회)")
print()
print(f"💵 월 평균 비용:       {total_cost_mean:,.0f}원")
print(f"   (±{total_cost_std:,.0f}원)")
print()
print(f"예상 범위:")
print(f"  최소: {total_cost_mean - total_cost_std:,.0f}원")
print(f"  평균: {total_cost_mean:,.0f}원")
print(f"  최대: {total_cost_mean + total_cost_std:,.0f}원")
print()
print(f"💳 연간 예상 비용:     {total_cost_mean * 12:,.0f}원")
print(f"                       ({total_cost_mean * 12 / 10000:.1f}만원)")

💰 한 달 LLM 비용 계산
월 평균 케이스:        5,840.5건
1건당 질문 수:         1번
1질문당 비용:          0.6543503999999999원

월 총 질문 수:         5,840.5회
                       (±651.9회)

💵 월 평균 비용:       3,822원
   (±427원)

예상 범위:
  최소: 3,395원
  평균: 3,822원
  최대: 4,248원

💳 연간 예상 비용:     45,861원
                       (4.6만원)


In [28]:
# retriever.invoke("1월에서 2월 사이 투썸플레이스 키오스크")

In [29]:
# %%time
# question = "투썸 매장에서 영수증프린터 오류 어떻게 처리했어?"
# response = chain.invoke(question)
# print(f"💬 답변:\n{response}")

In [30]:
# # 체인 실행(Run Chain)
# # 문서에 대한 질의를 입력하고, 답변을 출력합니다.
# question = "'투썸플레이스', '키오스크' 해결 사례?"
# response = chain.invoke(question)
# print(response)

In [31]:
# question = "1월에서 2월 사이의 투썸플레이스 영수증프린터 문의사항을 5개 보여줘"
# response = chain.invoke(question)
# print(response)